## PROJECT DOMAIN

Loan Default Prediction adalah proses memprediksi kemungkinan seorang peminjam gagal membayar kembali pinjaman berdasarkan data historis dan atribut tertentu. Hal ini biasanya dilakukan oleh institusi keuangan (bank, fintech) untuk mengurangi risiko kerugian finansial.

##PROBLEM STATEMENT

Di sektor keuangan, kemampuan untuk menilai kelayakan kredit peminjam secara akurat sangat penting untuk mengurangi risiko gagal bayar. Metode evaluasi tradisional seringkali kurang efektif dalam menangkap pola kompleks dalam data peminjam, yang dapat menyebabkan persetujuan terhadap peminjam berisiko tinggi atau penolakan terhadap peminjam yang sebenarnya layak.

Masalah utama yang ingin diselesaikan adalah bagaimana membangun model machine learning yang andal untuk memprediksi kemungkinan seorang peminjam gagal membayar pinjaman, berdasarkan data historis dan berbagai fitur penting seperti pendapatan, riwayat kredit, status pekerjaan, jumlah pinjaman, dan faktor sosio-ekonomi lainnya.

Tujuan akhirnya adalah membantu institusi keuangan dalam mengambil keputusan pinjaman yang lebih cerdas, mengurangi kerugian akibat kredit macet, dan meningkatkan efisiensi manajemen risiko kredit.

##GOALS

Memprediksi apakah peminjam akan gagal bayar (default) atau tidak.

Membantu lembaga keuangan mengambil keputusan kredit yang lebih akurat.

Mengurangi risiko kredit macet (Non-Performing Loans).

Mengoptimalkan proses penilaian risiko dan persetujuan pinjaman.

Meningkatkan efisiensi dan kecepatan dalam analisis kelayakan pinjaman.

##SOLUTION STATEMENT

Membangun model machine learning yang dapat memprediksi kemungkinan gagal bayar berdasarkan data historis peminjam.

Menggunakan fitur seperti pendapatan, riwayat kredit, pekerjaan, dan jumlah pinjaman untuk melatih model prediktif.

Menerapkan algoritma seperti Random Forest, XGBoost, atau Logistic Regression untuk hasil akurat dan andal.

Menyediakan sistem pendukung keputusan bagi lembaga keuangan untuk menyetujui atau menolak pinjaman secara otomatis dan efisien.

Mengintegrasikan visualisasi risiko untuk membantu memahami faktor-faktor utama yang mempengaruhi default.

##Data Understanding

##Import data dari kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

In [ ]:
!kaggle datasets download -d kmldas/loan-default-prediction

In [ ]:
!mkdir loan-default-prediction
!unzip loan-default-prediction.zip -d loan-default-prediction
!ls loan-default-prediction

##Import library yang dibutuhkan

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

##Exploratory data analysis

In [ ]:
df = pd.read_csv('/content/loan-default-prediction/Default_Fin.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
sns.countplot(x='Defaulted?', data=df)
plt.title("Distribusi Status Gagal Bayar")
plt.xlabel("Defaulted?")
plt.ylabel("Jumlah Peminjam")
plt.show()

In [ ]:
sns.countplot(x='Employed', hue='Defaulted?', data=df)
plt.title("Status Gagal Bayar Berdasarkan Status Pekerjaan")
plt.xlabel("Employed")
plt.ylabel("Jumlah")
plt.show()


In [ ]:
sns.boxplot(x='Defaulted?', y='Bank Balance', data=df)
plt.title("Distribusi Saldo Bank terhadap Status Gagal Bayar")
plt.xlabel("Defaulted?")
plt.ylabel("Bank Balance")
plt.show()


In [ ]:
sns.boxplot(x='Defaulted?', y='Annual Salary', data=df)
plt.title("Distribusi Gaji Tahunan terhadap Status Gagal Bayar")
plt.xlabel("Defaulted?")
plt.ylabel("Annual Salary")
plt.show()


In [ ]:
numeric_cols = ['Bank Balance', 'Annual Salary']
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm')
plt.title("Matriks Korelasi Fitur Numerik")
plt.show()


##Data Preparation

In [ ]:
X = df.drop(columns=["Defaulted?"])
y = df["Defaulted?"]

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [ ]:
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled,
                                                    y_encoded,
                                                    test_size=0.2,
                                                    random_state=42)

##Modeling

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
import tensorflow as tf

model = Sequential()
model.add(Input(shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(1))

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes = True)


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=55,
                    batch_size=10,
                    validation_data=(X_test, y_test))

##Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Akurasi Model: {test_accuracy:.4f}")
print(f"Loss Model: {test_loss:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
if "accuracy" in history.history and "val_accuracy" in history.history:
    ax.plot(history.history["accuracy"], label='Training Accuracy')
    ax.plot(history.history["val_accuracy"], label='Validation Accuracy')
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    ax.set_title('Training vs Validation Accuracy')
    ax.legend()
    plt.show()
else:
    print("Key 'accuracy' atau 'val_accuracy' tidak ditemukan dalam history.")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
if 'loss' in history.history:
    ax.plot(history.history['loss'], label='Training Loss')

if 'val_loss' in history.history:
    ax.plot(history.history['val_loss'], label='Validation Loss')

ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.set_title('Model Loss')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = y_test
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=le.classes_.astype(str), yticklabels=le.classes_.astype(str),
            linewidths=.5, annot_kws={"size": 12})
plt.xlabel("Predicted Label", fontsize=14)
plt.ylabel("True Label", fontsize=14)
plt.title("Confusion Matrix", fontsize=16)
plt.tight_layout()
plt.show()
print("\nClassification Report:\n")
print(classification_report(y_true_classes, y_pred_classes, target_names=le.classes_.astype(str)))

##Deployment

##Model simulation

In [ ]:
sample_input = np.array([[78, 42 ,42, 20.13, 81.60,	7.62, 262.71]])
sample_input_df = pd.DataFrame(sample_input)


In [ ]:
sample_data = [[78, 42, 42, 20.13]]

sample_data_reshaped = np.array(sample_data).reshape(1, -1)

In [ ]:
sample_data_scaled = scaler.transform(sample_data_reshaped)


In [ ]:
predicted_class = np.argmax(model.predict(sample_data_scaled))
predicted_status = le.inverse_transform([predicted_class])
print(f"Prediksi Status Pinjaman: {predicted_status[0]}")

In [ ]:
sample_data = [[78, 42, 42, 20.13, 81.60, 7.62, 262.71]]

sample_data_reshaped = np.array(sample_data).reshape(1, -1)

##Save Model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('loan_default_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import joblib
joblib.dump(le, 'label_encoder.pkl')
joblib.dump(scaler, 'loan_scaler.pkl')